In [ ]:
import os
os.chdir("../")

## Inference of MPT on WikiTQ

In [ ]:
import torch
import torch.nn as nn

from src import MPTModelForGenerativeQuestionAnswering
from utils import process_config, prepare_dataloaders

import json
import pandas as pd
from datasets import load_dataset
from tqdm import tqdm

In [ ]:
with open("configs/wiki_tq/mpt.json", "r") as f:
    config = json.load(f) 

In [ ]:
config = process_config(config)

In [ ]:
dataset = load_dataset(config.data.data_path)

In [ ]:
train_dataloader, validation_dataloader, test_dataloader, tokenizer = prepare_dataloaders(dataset, config)

In [ ]:
model = MPTModelForGenerativeQuestionAnswering(config)
model.load_state_dict(torch.load("logs/table_question_answering_mpt7b_standard/checkpoints/epoch=30.pt"))

In [ ]:
import os
import re
import sys
import argparse
import unicodedata
import numpy as np
import pandas as pd
from tqdm import tqdm
from codecs import open
from math import isnan, isinf
from easydict import EasyDict
from torch.utils.data import DataLoader
from abc import ABCMeta, abstractmethod

def normalize(x):

    if not isinstance(x, str):
        x = x.decode('utf8', errors='ignore')

    # Remove diacritics
    x = ''.join(c for c in unicodedata.normalize('NFKD', x)
                if unicodedata.category(c) != 'Mn')
    
    # Normalize quotes and dashes
    x = re.sub(r"[‘’´`]", "'", x)
    x = re.sub(r"[“”]", "\"", x)
    x = re.sub(r"[‐‑‒–—−]", "-", x)

    while True:
        
        old_x = x

        # Remove citations
        x = re.sub(r"((?<!^)\[[^\]]*\]|\[\d+\]|[•♦†‡*#+])*$", "", x.strip())
        
        # Remove details in parenthesis
        x = re.sub(r"(?<!^)( \([^)]*\))*$", "", x.strip())
        
        # Remove outermost quotation mark
        x = re.sub(r'^"([^"]*)"$', r'\1', x.strip())
        
        if x == old_x:
            break
    
    # Remove final '.'
    if x and x[-1] == '.':
        x = x[:-1]
    
    # Collapse whitespaces and convert to lower case
    x = re.sub(r'\s+', ' ', x, flags=re.U).lower().strip()
    
    return x


class Value(object):
    __metaclass__ = ABCMeta

    # Should be populated with the normalized string
    _normalized = None

    @abstractmethod
    def match(self, other):
        """Return True if the value matches the other value.

        Args:
            other (Value)
        Returns:
            a boolean
        """
        pass

    @property
    def normalized(self):
        return self._normalized


class StringValue(Value):

    def __init__(self, content):
        assert isinstance(content, str)
        self._normalized = normalize(content)
        self._hash = hash(self._normalized)

    def __eq__(self, other):
        return isinstance(other, StringValue) and self.normalized == other.normalized

    def __hash__(self):
        return self._hash

    def __str__(self):
        return 'S' + str([self.normalized])

    __repr__ = __str__

    def match(self, other):
        assert isinstance(other, Value)
        return self.normalized == other.normalized


class NumberValue(Value):

    def __init__(self, amount, original_string=None):
        assert isinstance(amount, (int, float))
        if abs(amount - round(amount)) < 1e-6:
            self._amount = int(amount)
        else:
            self._amount = float(amount)
        if not original_string:
            self._normalized = str(self._amount)
        else:
            self._normalized = normalize(original_string)
        self._hash = hash(self._amount)

    @property
    def amount(self):
        return self._amount

    def __eq__(self, other):
        return isinstance(other, NumberValue) and self.amount == other.amount

    def __hash__(self):
        return self._hash

    def __str__(self):
        return ('N(%f)' % self.amount) + str([self.normalized])

    __repr__ = __str__

    def match(self, other):
        assert isinstance(other, Value)
        if self.normalized == other.normalized:
            return True
        if isinstance(other, NumberValue):
            return abs(self.amount - other.amount) < 1e-6
        return False

    @staticmethod
    def parse(text):
        """Try to parse into a number.

        Return:
            the number (int or float) if successful; otherwise None.
        """
        try:
            return int(text)
        except:
            try:
                amount = float(text)
                assert not isnan(amount) and not isinf(amount)
                return amount
            except:
                return None


class DateValue(Value):

    def __init__(self, year, month, day, original_string=None):

        """Create a new DateValue. Placeholders are marked as -1."""
        assert isinstance(year, int)
        assert isinstance(month, int) and (month == -1 or 1 <= month <= 12)
        assert isinstance(day, int) and (day == -1 or 1 <= day <= 31)
        assert not (year == month == day == -1)
        
        self._year = year
        self._month = month
        self._day = day
        
        if not original_string:
            self._normalized = '{}-{}-{}'.format(
                year if year != -1 else 'xx',
                month if month != -1 else 'xx',
                day if day != '-1' else 'xx')
        else:
            self._normalized = normalize(original_string)
        
        self._hash = hash((self._year, self._month, self._day))

    @property
    def ymd(self):
        return (self._year, self._month, self._day)

    def __eq__(self, other):
        return isinstance(other, DateValue) and self.ymd == other.ymd

    def __hash__(self):
        return self._hash

    def __str__(self):
        return (('D(%d,%d,%d)' % (self._year, self._month, self._day))
                + str([self._normalized]))

    __repr__ = __str__

    def match(self, other):
        
        assert isinstance(other, Value)
        
        if self.normalized == other.normalized:
            return True
        
        if isinstance(other, DateValue):
            return self.ymd == other.ymd
        
        return False

    @staticmethod
    def parse(text):
        """Try to parse into a date.

        Return:
            tuple (year, month, date) if successful; otherwise None.
        """
        try:
            ymd = text.lower().split('-')
            assert len(ymd) == 3
            year = -1 if ymd[0] in ('xx', 'xxxx') else int(ymd[0])
            month = -1 if ymd[1] == 'xx' else int(ymd[1])
            day = -1 if ymd[2] == 'xx' else int(ymd[2])
            assert not (year == month == day == -1)
            assert month == -1 or 1 <= month <= 12
            assert day == -1 or 1 <= day <= 31
            return (year, month, day)
        except:
            return None


def to_value(original_string, corenlp_value=None):
    """Convert the string to Value object.

    Args:
        original_string (basestring): Original string
        corenlp_value (basestring): Optional value returned from CoreNLP
    Returns:
        Value
    """

    if isinstance(original_string, Value):
        # Already a Value
        return original_string
    
    if not corenlp_value:
        corenlp_value = original_string
    
    # Number?
    amount = NumberValue.parse(corenlp_value)
    
    if amount is not None:
        return NumberValue(amount, original_string)
    
    # Date?
    ymd = DateValue.parse(corenlp_value)
    if ymd is not None:
        if ymd[1] == ymd[2] == -1:
            return NumberValue(ymd[0], original_string)
        else:
            return DateValue(ymd[0], ymd[1], ymd[2], original_string)
    
    # String.
    return StringValue(original_string)


def to_value_list(original_strings, corenlp_values=None):
    """Convert a list of strings to a list of Values

    Args:
        original_strings (list[basestring])
        corenlp_values (list[basestring or None])
    Returns:
        list[Value]
    """
    assert isinstance(original_strings, (list, tuple, set))
    if corenlp_values is not None:
        assert isinstance(corenlp_values, (list, tuple, set))
        assert len(original_strings) == len(corenlp_values)
        return list(set(to_value(x, y) for (x, y)
                        in zip(original_strings, corenlp_values)))
    else:
        return list(set(to_value(x) for x in original_strings))


def check_denotation(target_values, predicted_values):
    """Return True if the predicted denotation is correct.

    Args:
        target_values (list[Value])
        predicted_values (list[Value])
    Returns:
        bool
    """
    
    # Check size
    if len(target_values) != len(predicted_values):
        return False
    
    # Check items
    for target in target_values:
        if not any(target.match(pred) for pred in predicted_values):
            return False
    
    return True


def tsv_unescape(x):
    """Unescape strings in the TSV file.
    Escaped characters include:
        newline (0x10) -> backslash + n
        vertical bar (0x7C) -> backslash + p
        backslash (0x5C) -> backslash + backslash

    Args:
        x (str or unicode)
    Returns:
        a unicode
    """
    return x.replace(r'\n', '\n').replace(r'\p', '|').replace('\\\\', '\\')


def tsv_unescape_list(x):
    """Unescape a list in the TSV file.
    List items are joined with vertical bars (0x5C)

    Args:
        x (str or unicode)
    Returns:
        a list of unicodes
    """
    return [tsv_unescape(y) for y in x.split('|')]

In [ ]:
class AverageMeter:
    def __init__(self):
        self.reset()

    def reset(self):
        self.count = 0
        self.sum = 0
        self.avg = 0

    def update(self, value, n=1):
        self.sum += value
        self.count += n
        self.avg = self.sum / self.count

metrics = {"accuracy": AverageMeter()}


In [ ]:
model.to("cuda:0")

In [ ]:
def compute_metrics(dataloader):

    with tqdm(dataloader, unit = "batch", position = 0, leave = True, desc = "Metrics") as tepoch:
        for batch_idx, batch in enumerate(tepoch):
            

            if config.model.use_position_ids:
                raise NotImplementedError
                input_ids, attention_mask, token_type_ids, decoder_input_ids, position_ids, labels = batch

                # model.module.generate(input_ids = input_ids, attention_mask = attention_mask, tokenizer = tokenizer)
                output_ids = model.model.generate(input_ids = input_ids.to(device), attention_mask = attention_mask.to(device), 
                                                        max_length = config.tokenizer.output_max_length, 
                                                        num_beams = 3, early_stopping = True, position_ids = position_ids.to(device))

            else:
                if config.model.type == "encoder-decoder":
                    input_ids, attention_mask, token_type_ids, decoder_input_ids, labels = batch
                    actual_output_ids = labels.clone()
                    output_ids = model.model.generate(input_ids = input_ids.to("cuda:0"), max_new_tokens = config.tokenizer.output_max_length, 
                                                            num_beams = 3, early_stopping = True, attention_mask = attention_mask.to("cuda:0")).detach().cpu()
                elif config.model.type == "decoder-only":
                    input_ids, attention_mask, token_type_ids, inference_input_ids, inference_attention_mask, actual_output_ids, labels = batch
                    output_ids = model.model.generate(input_ids = inference_input_ids.to("cuda:0"), max_new_tokens = config.tokenizer.output_max_length, 
                                                        num_beams = 3, early_stopping = True, attention_mask = inference_attention_mask.to("cuda:0")).detach().cpu()

            predicted_sequence = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
            if config.model.type == "decoder-only":
                for i in range(len(predicted_sequence)):
                    predicted_sequence[i] = predicted_sequence[i].replace(tokenizer.decode(inference_input_ids[i], skip_special_tokens = True), "")
            
            actual_sequence = tokenizer.batch_decode(actual_output_ids, skip_special_tokens = True)

            for a, p in zip(actual_sequence, predicted_sequence):
                pred = to_value_list([p])
                gold = to_value_list([a])

                print(pred)
                print(gold)
                exit(0)

                correct = check_denotation(pred, gold)

                if correct:
                    metrics["accuracy"].update(1, 1)
                else:
                    metrics["accuracy"].update(0, 1)

In [ ]:
compute_metrics(test_dataloader)

## Quality measurement of Bert Based decomposition

In [ ]:
import pandas as pd

In [ ]:
import pickle
with open("datasets/wiki_tq/test_soft_decomposition.pkl", "rb") as f:
    decomposed_table = pickle.load(f)

In [ ]:
from datasets import load_dataset

In [ ]:
dataset = load_dataset("wikitablequestions")

In [ ]:
def visualise_soft_decomposition(index):

    question = dataset["test"][index]["question"]
    answers = dataset["test"][index]["answers"]

    table_column_names = dataset["test"][index]["table"]["header"]
    table_content_values = dataset["test"][index]["table"]["rows"]
    table = pd.DataFrame.from_dict({str(col).lower(): [str(table_content_values[j][i]).lower() for j in range(len(table_content_values))] for i, col in enumerate(table_column_names)})
    
    table_decomposed = decomposed_table[index]
    
    print("\n\nQuestion: ", question)
    print("\n\Answer: ", answers)

    print("\n\nActual table: ")
    display(table)
    print("\n\nDecomposed table:")
    display(table_decomposed)

In [ ]:
visualise_soft_decomposition(0)

In [ ]:
visualise_soft_decomposition(1)

In [ ]:
visualise_soft_decomposition(2)

In [ ]:
visualise_soft_decomposition(3)

In [ ]:
visualise_soft_decomposition(4)

In [ ]:
visualise_soft_decomposition(5)

In [ ]:
visualise_soft_decomposition(6)

In [ ]:
visualise_soft_decomposition(7)

In [ ]:
visualise_soft_decomposition(8)

In [ ]:
visualise_soft_decomposition(9)

In [ ]:
visualise_soft_decomposition(10)

## Analysis of number of tables where answer is present

In [ ]:
from datasets import load_dataset
import pandas as pd

In [ ]:
dataset = load_dataset("wikitablequestions")

### Train

In [ ]:
train_dataset = dataset["train"]

In [ ]:
train_dataset[0]["answers"]

In [ ]:
train_dataset[0]["table"]["rows"]

In [ ]:
count = 0
total = 0

In [ ]:
from tqdm import tqdm

In [ ]:
for i in tqdm(range(len(train_dataset)), position = 0, leave = True, total = len(train_dataset)):
    answers = train_dataset[i]["answers"]
    rows = train_dataset[i]["table"]["rows"]
    flag = False 
    for row in rows:
        for ans in answers:
            if ans in row:
                flag = True

    if flag:
        count += 1
    total += 1

In [ ]:
total

In [ ]:
count

In [ ]:
test_dataset = dataset["test"]

In [ ]:
count = 0
total = 0

In [ ]:
for i in tqdm(range(len(test_dataset)), position = 0, leave = True, total = len(test_dataset)):
    answers = test_dataset[i]["answers"]
    rows = test_dataset[i]["table"]["rows"]
    flag = False 
    for row in rows:
        for ans in answers:
            if ans in row:
                flag = True

    if flag:
        count += 1
    total += 1

In [ ]:
total

In [ ]:
count

In [ ]:
val_dataset = dataset["validation"]

In [ ]:
count = 0
total = 0

In [ ]:
for i in tqdm(range(len(val_dataset)), position = 0, leave = True, total = len(val_dataset)):
    answers = val_dataset[i]["answers"]
    rows = val_dataset[i]["table"]["rows"]
    flag = False 
    for row in rows:
        for ans in answers:
            if ans in row:
                flag = True

    if flag:
        count += 1
    total += 1

In [ ]:
count

In [ ]:
total

In [ ]:
import pickle
from datasets import load_dataset

In [ ]:
dataset = load_dataset("wikitablequestions")

In [ ]:
train_dataset = dataset["train"]

In [ ]:
with open("datasets/wiki_tq/train_soft_decomposition.pkl", "rb") as f:
    train_soft_decomposition = pickle.load(f)

In [ ]:
from tqdm import tqdm

In [ ]:
count = 0
total = 0

In [ ]:
for i in tqdm(range(len(train_dataset)), position = 0, leave = True, total = len(train_dataset)):
    answers = train_dataset[i]["answers"]
    for i in range(len(answers)):
        answers[i] = str(answers[i]).lower()
    
    table = train_soft_decomposition[i]
    rows = table.values.tolist()
    flag = False 
    for row in rows:
        for ans in answers:
            if ans in row:
                flag = True

    if flag:
        count += 1
    total += 1

In [ ]:
count

In [ ]:
total

In [ ]:
test_dataset = dataset["test"]

In [ ]:
with open("datasets/wiki_tq/test_soft_decomposition.pkl", "rb") as f:
    test_soft_decomposition = pickle.load(f)

In [ ]:
count = 0
total = 0

In [ ]:
for i in tqdm(range(len(test_dataset)), position = 0, leave = True, total = len(test_dataset)):
    answers = test_dataset[i]["answers"]
    for i in range(len(answers)):
        answers[i] = str(answers[i]).lower()
    
    table = test_soft_decomposition[i]
    rows = table.values.tolist()
    flag = False 
    for row in rows:
        for ans in answers:
            if ans in row:
                flag = True

    if flag:
        count += 1
    total += 1

In [ ]:
total

In [ ]:
count

In [ ]:
validation_dataset = dataset["validation"]

In [ ]:
with open("datasets/wiki_tq/validation_soft_decomposition.pkl", "rb") as f:
    validation_soft_decomposition = pickle.load(f)

In [ ]:
count = 0
total = 0

In [ ]:
for i in tqdm(range(len(validation_dataset)), position = 0, leave = True, total = len(validation_dataset)):
    answers = validation_dataset[i]["answers"]
    for i in range(len(answers)):
        answers[i] = str(answers[i]).lower()
    
    table = validation_soft_decomposition[i]
    rows = table.values.tolist()
    flag = False 
    for row in rows:
        for ans in answers:
            if ans in row:
                flag = True

    if flag:
        count += 1
    total += 1

In [ ]:
count

In [ ]:
total

# DATER Analysis

In [ ]:
import json

In [ ]:
with open("dater_outputs/saved/results/wtq/gloc_wtq_end2end_wikitq_test.json", "r") as f:
    test_preds = json.load(f)

In [ ]:
len(test_preds)

In [ ]:
from typing import collections

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
u"""Official Evaluator for WikiTableQuestions Dataset
There are 3 value types
1. String (unicode)
2. Number (float)
3. Date (a struct with 3 fields: year, month, and date)
   Some fields (but not all) can be left unspecified. However, if only the year
   is specified, the date is automatically converted into a number.
Target denotation = a set of items
- Each item T is a raw unicode string from Mechanical Turk
- If T can be converted to a number or date (via Stanford CoreNLP), the
    converted value (number T_N or date T_D) is precomputed
Predicted denotation = a set of items
- Each item P is a string, a number, or a date
- If P is read from a text file, assume the following
  - A string that can be converted into a number (float) is converted into a
    number
  - A string of the form "yyyy-mm-dd" is converted into a date. Unspecified
    fields can be marked as "xx". For example, "xx-01-02" represents the date
    January 2nd of an unknown year.
  - Otherwise, it is kept as a string
The predicted denotation is correct if
1. The sizes of the target denotation and the predicted denotation are equal
2. Each item in the target denotation matches an item in the predicted
    denotation
A target item T matches a predicted item P if one of the following is true:
1. normalize(raw string of T) and normalize(string form of P) are identical.
   The normalize method performs the following normalizations on strings:
   - Remove diacritics (é → e)
   - Convert smart quotes (‘’´`“”) and dashes (‐‑‒–—−) into ASCII ones
   - Remove citations (trailing •♦†‡*#+ or [...])
   - Remove details in parenthesis (trailing (...))
   - Remove outermost quotation marks
   - Remove trailing period (.)
   - Convert to lowercase
   - Collapse multiple whitespaces and strip outermost whitespaces
2. T can be interpreted as a number T_N, P is a number, and P = T_N
3. T can be interpreted as a date T_D, P is a date, and P = T_D
   (exact match on all fields; e.g., xx-01-12 and 1990-01-12 do not match)
"""
__version__ = '1.0.2'

import sys, os, re, argparse
import unicodedata
from codecs import open
from math import isnan, isinf
from abc import ABCMeta, abstractmethod


################ String Normalization ################

def normalize(x):
    if not isinstance(x, str):
        x = x.decode('utf8', errors='ignore')
    # Remove diacritics
    x = ''.join(c for c in unicodedata.normalize('NFKD', x)
                if unicodedata.category(c) != 'Mn')
    # Normalize quotes and dashes
    x = re.sub(r"[‘’´`]", "'", x)
    x = re.sub(r"[“”]", "\"", x)
    x = re.sub(r"[‐‑‒–—−]", "-", x)
    while True:
        old_x = x
        # Remove citations
        x = re.sub(r"((?<!^)\[[^\]]*\]|\[\d+\]|[•♦†‡*#+])*$", "", x.strip())
        # Remove details in parenthesis
        x = re.sub(r"(?<!^)( \([^)]*\))*$", "", x.strip())
        # Remove outermost quotation mark
        x = re.sub(r'^"([^"]*)"$', r'\1', x.strip())
        if x == old_x:
            break
    # Remove final '.'
    if x and x[-1] == '.':
        x = x[:-1]
    # Collapse whitespaces and convert to lower case
    x = re.sub(r'\s+', ' ', x, flags=re.U).lower().strip()
    return x


################ Value Types ################

class Value(object):
    __metaclass__ = ABCMeta

    # Should be populated with the normalized string
    _normalized = None

    @abstractmethod
    def match(self, other):
        """Return True if the value matches the other value.
        Args:
            other (Value)
        Returns:
            a boolean
        """
        pass

    @property
    def normalized(self):
        return self._normalized


class StringValue(Value):

    def __init__(self, content):
        assert isinstance(content, str)
        self._normalized = normalize(content)
        self._hash = hash(self._normalized)

    def __eq__(self, other):
        return isinstance(other, StringValue) and self.normalized == other.normalized

    def __hash__(self):
        return self._hash

    def __str__(self):
        return 'S' + str([self.normalized])

    __repr__ = __str__

    def match(self, other):
        assert isinstance(other, Value)
        return self.normalized == other.normalized


class NumberValue(Value):

    def __init__(self, amount, original_string=None):
        assert isinstance(amount, (int, float))
        if abs(amount - round(amount)) < 1e-6:
            self._amount = int(amount)
        else:
            self._amount = float(amount)
        if not original_string:
            self._normalized = str(self._amount)
        else:
            self._normalized = normalize(original_string)
        self._hash = hash(self._amount)

    @property
    def amount(self):
        return self._amount

    def __eq__(self, other):
        return isinstance(other, NumberValue) and self.amount == other.amount

    def __hash__(self):
        return self._hash

    def __str__(self):
        return ('N(%f)' % self.amount) + str([self.normalized])

    __repr__ = __str__

    def match(self, other):
        assert isinstance(other, Value)
        if self.normalized == other.normalized:
            return True
        if isinstance(other, NumberValue):
            return abs(self.amount - other.amount) < 1e-6
        return False

    @staticmethod
    def parse(text):
        """Try to parse into a number.
        Return:
            the number (int or float) if successful; otherwise None.
        """
        try:
            return int(text)
        except:
            try:
                amount = float(text)
                assert not isnan(amount) and not isinf(amount)
                return amount
            except:
                return None


class DateValue(Value):

    def __init__(self, year, month, day, original_string=None):
        """Create a new DateValue. Placeholders are marked as -1."""
        assert isinstance(year, int)
        assert isinstance(month, int) and (month == -1 or 1 <= month <= 12)
        assert isinstance(day, int) and (day == -1 or 1 <= day <= 31)
        assert not (year == month == day == -1)
        self._year = year
        self._month = month
        self._day = day
        if not original_string:
            self._normalized = '{}-{}-{}'.format(
                year if year != -1 else 'xx',
                month if month != -1 else 'xx',
                day if day != '-1' else 'xx')
        else:
            self._normalized = normalize(original_string)
        self._hash = hash((self._year, self._month, self._day))

    @property
    def ymd(self):
        return (self._year, self._month, self._day)

    def __eq__(self, other):
        return isinstance(other, DateValue) and self.ymd == other.ymd

    def __hash__(self):
        return self._hash

    def __str__(self):
        return (('D(%d,%d,%d)' % (self._year, self._month, self._day))
                + str([self._normalized]))

    __repr__ = __str__

    def match(self, other):
        assert isinstance(other, Value)
        if self.normalized == other.normalized:
            return True
        if isinstance(other, DateValue):
            return self.ymd == other.ymd
        return False

    @staticmethod
    def parse(text):
        """Try to parse into a date.
        Return:
            tuple (year, month, date) if successful; otherwise None.
        """
        try:
            ymd = text.lower().split('-')
            assert len(ymd) == 3
            year = -1 if ymd[0] in ('xx', 'xxxx') else int(ymd[0])
            month = -1 if ymd[1] == 'xx' else int(ymd[1])
            day = -1 if ymd[2] == 'xx' else int(ymd[2])
            assert not (year == month == day == -1)
            assert month == -1 or 1 <= month <= 12
            assert day == -1 or 1 <= day <= 31
            return (year, month, day)
        except:
            return None


################ Value Instantiation ################

def to_value(original_string, corenlp_value=None):
    """Convert the string to Value object.
    Args:
        original_string (basestring): Original string
        corenlp_value (basestring): Optional value returned from CoreNLP
    Returns:
        Value
    """
    if isinstance(original_string, Value):
        # Already a Value
        return original_string
    if not corenlp_value:
        corenlp_value = original_string
    # Number?
    amount = NumberValue.parse(corenlp_value)
    if amount is not None:
        return NumberValue(amount, original_string)
    # Date?
    ymd = DateValue.parse(corenlp_value)
    if ymd is not None:
        if ymd[1] == ymd[2] == -1:
            return NumberValue(ymd[0], original_string)
        else:
            return DateValue(ymd[0], ymd[1], ymd[2], original_string)
    # String.
    return StringValue(original_string)


def to_value_list(original_strings, corenlp_values=None):
    """Convert a list of strings to a list of Values
    Args:
        original_strings (list[basestring])
        corenlp_values (list[basestring or None])
    Returns:
        list[Value]
    """
    assert isinstance(original_strings, (list, tuple, set))
    if corenlp_values is not None:
        assert isinstance(corenlp_values, (list, tuple, set))
        assert len(original_strings) == len(corenlp_values)
        return list(set(to_value(x, y) for (x, y)
                        in zip(original_strings, corenlp_values)))
    else:
        return list(set(to_value(x) for x in original_strings))


################ Check the Predicted Denotations ################

def check_denotation(target_values, predicted_values):
    """Return True if the predicted denotation is correct.

    Args:
        target_values (list[Value])
        predicted_values (list[Value])
    Returns:
        bool
    """
    # Check size
    if len(target_values) != len(predicted_values):
        return False
    # Check items
    for target in target_values:
        if not any(target.match(pred) for pred in predicted_values):
            return False
    return True


################ Batch Mode ################

def tsv_unescape(x):
    """Unescape strings in the TSV file.
    Escaped characters include:
        newline (0x10) -> backslash + n
        vertical bar (0x7C) -> backslash + p
        backslash (0x5C) -> backslash + backslash
    Args:
        x (str or unicode)
    Returns:
        a unicode
    """
    return x.replace(r'\n', '\n').replace(r'\p', '|').replace('\\\\', '\\')


def tsv_unescape_list(x):
    """Unescape a list in the TSV file.
    List items are joined with vertical bars (0x5C)
    Args:
        x (str or unicode)
    Returns:
        a list of unicodes
    """
    return [tsv_unescape(y) for y in x.split('|')]


def main():
    parser = argparse.ArgumentParser()
    parser.add_argument('--tagged-dataset-path',
                        default='../../datasets/wtq/raw_dataset/wtq/tagged/data',
                        help='Directory containing CoreNLP-tagged dataset TSV file')
    parser.add_argument('--validation-ids-path',
                        default='omnitab_download/wtq/validation_ids.txt',
                        help='File containing ids of the validation set.')
    parser.add_argument('--split', default='validation',
                        help='On which split to evaluate prediction_path. Could be "test" or "validation"')
    parser.add_argument('--separator', default=', ',
                        help='Separator used to concat multiple answer entities')
    parser.add_argument('--prediction_path',default='gloc_wtq_end2end_wikitq_validation_all.json',
                        help='Path to the prediction file. Each line contains one prediction')
    args = parser.parse_args()

    # ID string --> list[Value]
    target_values_map = {}
    for filename in os.listdir(args.tagged_dataset_path):
        if filename[0]=='.':
            continue
        filename = os.path.join(args.tagged_dataset_path, filename)
        print('Reading dataset from', filename)
        with open(filename, 'r', 'utf8') as fin:
            header = fin.readline().rstrip('\n').split('\t')
            for line in fin:
                stuff = dict(zip(header, line.rstrip('\n').split('\t')))
                ex_id = stuff['id']
                original_strings = tsv_unescape_list(stuff['targetValue'])
                canon_strings = tsv_unescape_list(stuff['targetCanon'])

                target_values_map[ex_id] = to_value_list(
                    original_strings, canon_strings)
    # validation_ids = list(map(lambda x: x.strip(), open(args.validation_ids_path, 'r')))
    # print('Read', len(target_values_map), 'examples')
    # print(target_values_map)
    st2id = {}
    import json
    import collections
    import numpy as np
    if args.split == 'test':
        with open(os.path.join('../../datasets/wtq','test_lower.jsonl')) as f:
            lines = f.readlines()
            for l in lines:
                dic = json.loads(l)
                st = dic['statement']
                ids = dic['ids']
                st2id[st] = ids
        with open('gloc_wtq_end2end_wikitq_test.json','r') as f:
            dic = json.load(f)
        print('test')
    if args.split == 'validation':
        with open(os.path.join('../../datasets/wtq','valid_lower.jsonl')) as f:
            lines = f.readlines()
            for l in lines:
                dic = json.loads(l)
                st = dic['statement']
                ids = dic['ids']
                st2id[st] = ids
        with open('gloc_wtq_end2end_wikitq_validation.json','r') as f:
            dic = json.load(f)
        print('val')
        # with open('gloc_wtq_end2end_wikitq_validation_wo_sql.json','r') as f:
        #     dic = json.load(f)
        

        
    num_examples, num_correct = 0, 0
    simple_right = 0
    simple = 0
    cmplex_right = 0

    cmplex = 0
    large = 0
    small = 0
    deno_acc = 0
    for key in dic:
        to_union = collections.defaultdict(float)
        it = dic[key]
        table = it['data_item']['table_text']
        st = it['data_item']['statement']
        ######### col filed################
        preds = it['generations']
        
        for pred in preds:
            log_prob_mean = pred[2]
            pred = pred[0]

            pred = pred.split('therefore,the answer is :')[-1]
            
            key = pred
            to_union[key] += np.exp(log_prob_mean)
        d_ordered = sorted(to_union.items(),key=lambda x:x[1],reverse=True)
        try:
            pred_answer = d_ordered[0][0].split('\n')[0].strip()
        except Exception:
            pred_answer = 'error'
        st = st.split('\n')[0]
        target_values = target_values_map[st2id[st]]
        # treat pred as an single entity because of 2 prompt with out example
        # if len(target_values) == 1:
        #     pred_answer = [pred_answer]
        # else:
        #     pred_answer = pred_answer.split(',')
        
        # treat pred answer as only on entity
        pred_answer = [pred_answer]
        pred_answer = to_value_list(pred_answer)
        # print(pred_answer,target_values)
        # if len(st.split(' '))>=10:
        #     cmplex += 1
        #     if check_denotation(target_values,pred_answer):
        #         cmplex_right += 1
        # else:
        #     simple += 1
        #     if check_denotation(target_values,pred_answer):
        #         simple_right += 1
        if check_denotation(target_values,pred_answer):
            deno_acc += 1 
    print('Denotation Accuracy',deno_acc/len(dic))
    # print(len(dic))
    # print(simple)
    # print(cmplex)
    # print('s',simple_right/simple)
    # print('c',cmplex_right/cmplex)
#     print('Reading predictions from', args.prediction_path)
#     num_examples, num_correct = 0, 0
#     with open(args.prediction_path, 'r', 'utf8') as fin:
#         for lid, line in enumerate(fin):
#             if args.split == 'test':
#                 ex_id = f'nu-{lid}'
#             elif args.split == 'validation':
#                 ex_id = validation_ids[lid]
#             else:
#                 raise NotImplementedError
#             line = line.rstrip('\n').strip()

#             # treat each as single-entity or multi-entity answer based on the ground truth
#             # if len(target_values_map[ex_id]) == 1:
#             #     predictions = [line]
#             # else:
#             #     predictions = line.split(args.separator)
#             predictions = [line]
#             if ex_id not in target_values_map:
#                 print
#                 'WARNING: Example ID "%s" not found' % ex_id
#             else:
#                 target_values = target_values_map[ex_id]
#                 predicted_values = to_value_list(predictions)
#                 correct = check_denotation(target_values, predicted_values)
#                 print
#                 u'%s\t%s\t%s\t%s' % (ex_id, correct,
#                                      target_values, predicted_values)
#                 num_examples += 1
#                 if correct:
#                     num_correct += 1
#     print('Examples:', num_examples)
#     print('Correct:', num_correct)
#     print('Accuracy:', round((num_correct + 1e-9) / (num_examples + 1e-9), 4))

In [ ]:
import numpy as np

In [ ]:
count = 0
total = 0

In [ ]:
tables_list = []
questions_list = []
actual_answers_list = []
predicted_answers_list = []

In [ ]:
import pandas as pd
from tqdm import tqdm

In [ ]:
def get_error_files_instances(dic):
    for i, key in tqdm(enumerate(dic), position = 0, leave = True):
        to_union = collections.defaultdict(float)
        it = dic[key]
        table = it['data_item']['table_text']

        headers = table[0]
        rows = table[1:]
        question = it["data_item"]["statement"]
        table = pd.DataFrame.from_dict({str(col).lower(): [str(rows[j][i]).lower() for j in range(len(rows))] for i, col in enumerate(headers)})

        # display(table)

        st = it['data_item']['statement']
        ######### col filed################
        preds = it['generations']

        actual_answer = it["data_item"]["answer"]
        rows = table.values.tolist()
        flag = False 
        for row in rows:
            for ans in actual_answer:
                if ans.lower() in row:
                    flag = True
        
        # total += 1
        
        for pred in preds:
            log_prob_mean = pred[2]
            pred = pred[0]

            pred = pred.split('therefore,the answer is :')[-1]
            
            key = pred
            to_union[key] += np.exp(log_prob_mean)
        d_ordered = sorted(to_union.items(),key=lambda x:x[1],reverse=True)
        try:
            pred_answer = d_ordered[0][0].split('\n')[0].strip()
        except Exception:
            pred_answer = 'error'
        st = st.split('\n')[0]

        if not flag:
            # count += 1
            tables_list.append(table)
            questions_list.append(question)
            actual_answers_list.append(actual_answer)
            predicted_answers_list.append(pred_answer)

        # print(pred_answer)
        # print(actual_answer)

        # print(table)
        # if i == 0:
        #     break

In [ ]:
get_error_files_instances(test_preds)

In [ ]:
def print_data(index):

    table = tables_list[index]
    question = questions_list[index]
    actual_answer = actual_answers_list[index]
    predicted_answer = predicted_answers_list[index]

    print("\n\nQuestion: ", question)
    print("\n\nActual Answer: ", actual_answer)
    print("\n\nPredicted Answer: ", predicted_answer)
    print("\n\nTable")
    display(table)

In [ ]:
print_data(2)

In [ ]:
print_data(20)

In [ ]:
print_data(22)

In [ ]:
print_data(25)

In [ ]:
print_data(50)

In [ ]:
print_data(54)

In [ ]:
print_data(101)

In [ ]:
print_data(106)

In [ ]:
print_data(109)

In [ ]:
print_data(122)

# Wiki TQ Analysis

In [ ]:
from datasets import load_dataset

In [ ]:
dataset = load_dataset("wikitablequestions")

In [ ]:
train_dataset = dataset["train"]

In [ ]:
from tqdm import tqdm

In [ ]:
import pandas as pd

In [ ]:
tables_list = []
questions_list = []
answers_list = []


In [ ]:
for i in tqdm(range(len(train_dataset)), position = 0, leave = True, total = len(train_dataset)):
    question = train_dataset[i]["question"]
    answers = train_dataset[i]["answers"]
    rows = train_dataset[i]["table"]["rows"]
    headers = train_dataset[i]["table"]["header"]
    flag = False 
    for row in rows:
        for ans in answers:
            if ans in row:
                flag = True

    if not flag:
        table = pd.DataFrame.from_dict({str(col).lower(): [str(rows[j][i]).lower() for j in range(len(rows))] for i, col in enumerate(headers)})

        tables_list.append(table)
        questions_list.append(question)
        answers_list.append(answers)
    # if flag:
    #     count += 1
    # total += 1

In [ ]:
len(tables_list)

In [ ]:
def print_data(index):
    question = questions_list[index]
    table = tables_list[index]
    answers = answers_list[index]

    print("\n\nQuestion: ", question)
    print("\n\nAnswers", answers)
    print("\n\nTable:")
    display(table)

In [ ]:
print_data(0)

In [ ]:
print_data(2)

In [ ]:
print_data(3)

In [ ]:
print_data(4)

In [ ]:
print_data(6)

In [ ]:
print_data(7)

In [ ]:
print_data(20)

In [ ]:
print_data(27)

In [ ]:
print_data(50)

In [ ]:
print_data(146)

In [ ]:
print_data(172)

In [ ]:
print_data(188)

In [ ]:
print_data(189)

In [ ]:
print_data(191)

In [ ]:
print_data(196)

In [ ]:
print_data(199)

# Number Verbalisation

In [ ]:
import re

In [ ]:
def expand_numbers(strings):
    # Define lookup tables for various place values
    units = ['', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine']
    tens = ['', '', 'twenty', 'thirty', 'forty', 'fifty', 'sixty', 'seventy', 'eighty', 'ninety']
    teens = ['ten', 'eleven', 'twelve', 'thirteen', 'fourteen', 'fifteen', 'sixteen', 'seventeen', 'eighteen', 'nineteen']
    scales = ['', 'thousand', 'million', 'billion', 'trillion', 'quadrillion', 'quintillion']

    expanded_strings = []
    for string in strings:
        match = re.search(r'\d{1,3}(?:,\d{3})*|\d+', string)  # Regular expression to match numbers
        if match:
            number = match.group()
            number = number.replace(',', '')  # Remove commas if present
            number = int(number)
            if number == 0:
                expanded_string = 'zero'
            else:
                num_parts = []
                scale_count = 0
                while number > 0:
                    num_part = number % 1000  # Consider the last 3 digits
                    if num_part > 0:
                        num_part_words = []
                        hundreds_digit = num_part // 100
                        if hundreds_digit > 0:
                            num_part_words.append(units[hundreds_digit])
                            num_part_words.append('hundred')
                        tens_digit = (num_part // 10) % 10
                        units_digit = num_part % 10
                        if tens_digit == 1:
                            num_part_words.append(teens[units_digit])
                        else:
                            if tens_digit > 1:
                                num_part_words.append(tens[tens_digit])
                            if units_digit > 0:
                                num_part_words.append(units[units_digit])
                        num_parts.append(' '.join(num_part_words) + ' ' + scales[scale_count])
                    number //= 1000
                    scale_count += 1
                num_parts.reverse()
                expanded_string = ' '.join(num_parts)
            expanded_string = string.replace(match.group(), expanded_string)  # Replace number in string
            expanded_strings.append(expanded_string)
        else:
            expanded_strings.append(string)
    return expanded_strings

# Example usage
strings = ['The number is 145216.', 'Another example: 100,000.', 'No number in this string.']
expanded_strings = expand_numbers(strings)
for string in expanded_strings:
    print(string)


In [ ]:
def expand_numbers(strings):
    # Define lookup tables for various place values
    units = ['', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine']
    tens = ['', '', 'twenty', 'thirty', 'forty', 'fifty', 'sixty', 'seventy', 'eighty', 'ninety']
    teens = ['ten', 'eleven', 'twelve', 'thirteen', 'fourteen', 'fifteen', 'sixteen', 'seventeen', 'eighteen', 'nineteen']
    scales = ['', 'thousand', 'lakh', 'crore']

    expanded_strings = []
    for string in strings:
        match = re.search(r'\d{1,10}(?:,\d{3})*|\d+', string)  # Regular expression to match numbers
        print(match)
        if match:
            number = match.group()
            number = number.replace(',', '')  # Remove commas if present
            number = int(number)
            if number == 0:
                expanded_string = 'zero'
            else:
                num_parts = []
                scale_count = 0
                while number > 0:
                    num_part = number % 1000  # Consider the last 3 digits
                    if num_part > 0:
                        num_part_words = []
                        hundreds_digit = num_part // 100
                        if hundreds_digit > 0:
                            num_part_words.append(units[hundreds_digit])
                            num_part_words.append('hundred')
                        tens_digit = (num_part // 10) % 10
                        units_digit = num_part % 10
                        if tens_digit == 1:
                            num_part_words.append(teens[units_digit])
                        else:
                            if tens_digit > 1:
                                num_part_words.append(tens[tens_digit])
                            if units_digit > 0:
                                num_part_words.append(units[units_digit])
                        num_parts.append(' '.join(num_part_words) + ' ' + scales[scale_count])
                    number //= 1000
                    scale_count += 1
                num_parts.reverse()
                expanded_string = ' '.join(num_parts)
            expanded_string = string.replace(match.group(), expanded_string)  # Replace number in string
            expanded_strings.append(expanded_string)
        else:
            expanded_strings.append(string)
    return expanded_strings

# Example usage
strings = ['The number is 10207.', 'Another example: 100,000.', 'No number in this string.']
expanded_strings = expand_numbers(strings)
for string in expanded_strings:
    print(string)


# Differentiable decomposition analysis

In [ ]:
from src import EEDBartModelForGenerativeQuestionAnswering
from datasets import load_dataset
import json
from utils import process_config
from data import WikiTQDataset
import torch

In [ ]:
with open("configs/wiki_tq/tapex.json", "r") as f:
    config = json.load(f)

config = process_config(config)

In [ ]:
dataset = load_dataset(config.data.data_path)
test_dataset = WikiTQDataset(dataset, config, "test")

In [ ]:
model = EEDBartModelForGenerativeQuestionAnswering(config)
model.load_state_dict(torch.load("logs/table_question_answering_tapex_eed_soft_decomposition_epochs30/checkpoints/epoch=15.pt"))

In [ ]:
tokenizer = test_dataset.tokenizer

In [ ]:
model.to("cuda:0")

In [ ]:
def predict(index):

    input_ids, attention_mask, token_type_ids, decoder_input_ids, labels = test_dataset.__getitem__(index)
    actual_output_ids = labels.clone()
    

    token_scores = model.model.model.sigmoid(model.model.model.token_classifier(model.model.model.decomposer(input_ids = input_ids.unsqueeze(0).to("cuda:0"), attention_mask = attention_mask.unsqueeze(0).to("cuda:0"))[0]))

    print(tokenizer.decode(input_ids.squeeze(0), skip_special_tokens=True))
    print("\n\n")

    input_ids = input_ids.detach().cpu()[token_scores.squeeze().detach().cpu() > 0.93]
    print(tokenizer.decode(input_ids.squeeze(0), skip_special_tokens=True))
    # print(token_scores)
    # output_ids = model.model.generate(input_ids = input_ids.unsqueeze(0).to("cuda:0"), max_new_tokens = config.tokenizer.output_max_length, 
    #                                                         num_beams = 3, early_stopping = True, attention_mask = attention_mask.unsqueeze(0).to("cuda:0"))

    # predicted_sequence = tokenizer.decode(output_ids.squeeze(), skip_special_tokens=True)
    # actual_sequence = tokenizer.decode(actual_output_ids, skip_special_tokens = True)

    # print(predicted_sequence)
    # print(actual_sequence)

In [ ]:
predict(3)

In [ ]:
predict(4)

In [ ]:
predict(10)

In [ ]:
predict(20)

In [ ]:
predict(15)

In [ ]:
predict(23)

In [ ]:
predict(27)

In [ ]:
predict(12)

In [ ]:
predict(100)

# Question + Irrelevant table - anwers analysis

In [ ]:
from src import BartModelForGenerativeQuestionAnswering
from data import WikiTQDataset
from utils import process_config
from datasets import load_dataset
import json

In [ ]:
with open("configs/wiki_tq/tapex.json", "r") as f:
    config = json.load(f)
config = process_config(config)

In [ ]:
dataset = load_dataset(config.data.data_path)

In [ ]:
train_dataset = WikiTQDataset(dataset, config, "train")

In [ ]:
model = BartModelForGenerativeQuestionAnswering(config)

In [ ]:
device = "cuda:0"

In [ ]:
model.to(device)

In [ ]:
tokenizer = train_dataset.tokenizer

In [ ]:
from abc import ABCMeta, abstractmethod
import unicodedata
import re

def normalize(x):

    if not isinstance(x, str):
        x = x.decode('utf8', errors='ignore')

    # Remove diacritics
    x = ''.join(c for c in unicodedata.normalize('NFKD', x)
                if unicodedata.category(c) != 'Mn')
    
    # Normalize quotes and dashes
    x = re.sub(r"[‘’´`]", "'", x)
    x = re.sub(r"[“”]", "\"", x)
    x = re.sub(r"[‐‑‒–—−]", "-", x)

    while True:
        
        old_x = x

        # Remove citations
        x = re.sub(r"((?<!^)\[[^\]]*\]|\[\d+\]|[•♦†‡*#+])*$", "", x.strip())
        
        # Remove details in parenthesis
        x = re.sub(r"(?<!^)( \([^)]*\))*$", "", x.strip())
        
        # Remove outermost quotation mark
        x = re.sub(r'^"([^"]*)"$', r'\1', x.strip())
        
        if x == old_x:
            break
    
    # Remove final '.'
    if x and x[-1] == '.':
        x = x[:-1]
    
    # Collapse whitespaces and convert to lower case
    x = re.sub(r'\s+', ' ', x, flags=re.U).lower().strip()
    
    return x


class Value(object):
    __metaclass__ = ABCMeta

    # Should be populated with the normalized string
    _normalized = None

    @abstractmethod
    def match(self, other):
        """Return True if the value matches the other value.

        Args:
            other (Value)
        Returns:
            a boolean
        """
        pass

    @property
    def normalized(self):
        return self._normalized


class StringValue(Value):

    def __init__(self, content):
        assert isinstance(content, str)
        self._normalized = normalize(content)
        self._hash = hash(self._normalized)

    def __eq__(self, other):
        return isinstance(other, StringValue) and self.normalized == other.normalized

    def __hash__(self):
        return self._hash

    def __str__(self):
        return 'S' + str([self.normalized])

    __repr__ = __str__

    def match(self, other):
        assert isinstance(other, Value)
        return self.normalized == other.normalized


class NumberValue(Value):

    def __init__(self, amount, original_string=None):
        assert isinstance(amount, (int, float))
        if abs(amount - round(amount)) < 1e-6:
            self._amount = int(amount)
        else:
            self._amount = float(amount)
        if not original_string:
            self._normalized = str(self._amount)
        else:
            self._normalized = normalize(original_string)
        self._hash = hash(self._amount)

    @property
    def amount(self):
        return self._amount

    def __eq__(self, other):
        return isinstance(other, NumberValue) and self.amount == other.amount

    def __hash__(self):
        return self._hash

    def __str__(self):
        return ('N(%f)' % self.amount) + str([self.normalized])

    __repr__ = __str__

    def match(self, other):
        assert isinstance(other, Value)
        if self.normalized == other.normalized:
            return True
        if isinstance(other, NumberValue):
            return abs(self.amount - other.amount) < 1e-6
        return False

    @staticmethod
    def parse(text):
        """Try to parse into a number.

        Return:
            the number (int or float) if successful; otherwise None.
        """
        try:
            return int(text)
        except:
            try:
                amount = float(text)
                assert not isnan(amount) and not isinf(amount)
                return amount
            except:
                return None


class DateValue(Value):

    def __init__(self, year, month, day, original_string=None):

        """Create a new DateValue. Placeholders are marked as -1."""
        assert isinstance(year, int)
        assert isinstance(month, int) and (month == -1 or 1 <= month <= 12)
        assert isinstance(day, int) and (day == -1 or 1 <= day <= 31)
        assert not (year == month == day == -1)
        
        self._year = year
        self._month = month
        self._day = day
        
        if not original_string:
            self._normalized = '{}-{}-{}'.format(
                year if year != -1 else 'xx',
                month if month != -1 else 'xx',
                day if day != '-1' else 'xx')
        else:
            self._normalized = normalize(original_string)
        
        self._hash = hash((self._year, self._month, self._day))

    @property
    def ymd(self):
        return (self._year, self._month, self._day)

    def __eq__(self, other):
        return isinstance(other, DateValue) and self.ymd == other.ymd

    def __hash__(self):
        return self._hash

    def __str__(self):
        return (('D(%d,%d,%d)' % (self._year, self._month, self._day))
                + str([self._normalized]))

    __repr__ = __str__

    def match(self, other):
        
        assert isinstance(other, Value)
        
        if self.normalized == other.normalized:
            return True
        
        if isinstance(other, DateValue):
            return self.ymd == other.ymd
        
        return False

    @staticmethod
    def parse(text):
        """Try to parse into a date.

        Return:
            tuple (year, month, date) if successful; otherwise None.
        """
        try:
            ymd = text.lower().split('-')
            assert len(ymd) == 3
            year = -1 if ymd[0] in ('xx', 'xxxx') else int(ymd[0])
            month = -1 if ymd[1] == 'xx' else int(ymd[1])
            day = -1 if ymd[2] == 'xx' else int(ymd[2])
            assert not (year == month == day == -1)
            assert month == -1 or 1 <= month <= 12
            assert day == -1 or 1 <= day <= 31
            return (year, month, day)
        except:
            return None


def to_value(original_string, corenlp_value=None):
    """Convert the string to Value object.

    Args:
        original_string (basestring): Original string
        corenlp_value (basestring): Optional value returned from CoreNLP
    Returns:
        Value
    """

    if isinstance(original_string, Value):
        # Already a Value
        return original_string
    
    if not corenlp_value:
        corenlp_value = original_string
    
    # Number?
    amount = NumberValue.parse(corenlp_value)
    
    if amount is not None:
        return NumberValue(amount, original_string)
    
    # Date?
    ymd = DateValue.parse(corenlp_value)
    if ymd is not None:
        if ymd[1] == ymd[2] == -1:
            return NumberValue(ymd[0], original_string)
        else:
            return DateValue(ymd[0], ymd[1], ymd[2], original_string)
    
    # String.
    return StringValue(original_string)


def to_value_list(original_strings, corenlp_values=None):
    """Convert a list of strings to a list of Values

    Args:
        original_strings (list[basestring])
        corenlp_values (list[basestring or None])
    Returns:
        list[Value]
    """
    assert isinstance(original_strings, (list, tuple, set))
    if corenlp_values is not None:
        assert isinstance(corenlp_values, (list, tuple, set))
        assert len(original_strings) == len(corenlp_values)
        return list(set(to_value(x, y) for (x, y)
                        in zip(original_strings, corenlp_values)))
    else:
        return list(set(to_value(x) for x in original_strings))


def check_denotation(target_values, predicted_values):
    """Return True if the predicted denotation is correct.

    Args:
        target_values (list[Value])
        predicted_values (list[Value])
    Returns:
        bool
    """
    
    # Check size
    if len(target_values) != len(predicted_values):
        return False
    
    # Check items
    for target in target_values:
        if not any(target.match(pred) for pred in predicted_values):
            return False
    
    return True


def tsv_unescape(x):
    """Unescape strings in the TSV file.
    Escaped characters include:
        newline (0x10) -> backslash + n
        vertical bar (0x7C) -> backslash + p
        backslash (0x5C) -> backslash + backslash

    Args:
        x (str or unicode)
    Returns:
        a unicode
    """
    return x.replace(r'\n', '\n').replace(r'\p', '|').replace('\\\\', '\\')


def tsv_unescape_list(x):
    """Unescape a list in the TSV file.
    List items are joined with vertical bars (0x5C)

    Args:
        x (str or unicode)
    Returns:
        a list of unicodes
    """
    return [tsv_unescape(y) for y in x.split('|')]

In [ ]:
def evaluate(index):
    input_ids, attention_mask, token_type_ids, decoder_input_ids, labels = train_dataset.__getitem__(index)

    actual_output_ids = labels.clone()
    output_ids = model.model.generate(input_ids = input_ids.unsqueeze(0).to(device), max_new_tokens = config.tokenizer.output_max_length, 
                                            num_beams = 3, early_stopping = True, attention_mask = attention_mask.unsqueeze(0).to(device))

    
    p = tokenizer.decode(output_ids.squeeze(0), skip_special_tokens=True)
    a = tokenizer.decode(actual_output_ids, skip_special_tokens = True)
    pred = to_value_list([p])
    gold = to_value_list([a])
    verdict = check_denotation(pred, gold)

    return verdict

In [ ]:
from tqdm import tqdm

### Accuracy with whole table as input (training set)

In [ ]:
correct = 0
total = 0

In [ ]:
for i in tqdm(range(train_dataset.__len__()), position = 0, leave = True, total = train_dataset.__len__()):
    verdict = evaluate(i)
    if verdict:
        correct += 1
    total += 1

In [ ]:
correct / total

### Accuracy with decomposed table as input (training set)

In [ ]:
correct = 0
total = 0

In [ ]:
for i in tqdm(range(train_dataset.__len__()), position = 0, leave = True, total = train_dataset.__len__()):
    verdict = evaluate(i)
    if verdict:
        correct += 1
    total += 1

In [ ]:
correct / total

### Accuracy with irrelevant decomposed table as input (training set)

In [ ]:
correct = 0
total = 0

In [ ]:
for i in tqdm(range(train_dataset.__len__()), position = 0, leave = True, total = train_dataset.__len__()):
    verdict = evaluate(i)
    if verdict:
        correct += 1
    total += 1

In [ ]:
correct / total

# Table Decomposition regularisation analysis

In [ ]:
from src import EEDBartModelForGenerativeQuestionAnswering
from datasets import load_dataset
import json
from utils import process_config
from data import WikiTQDataset
import torch

In [ ]:
with open("configs/wiki_tq/tapex.json", "r") as f:
    config = json.load(f)

config = process_config(config)

In [ ]:
dataset = load_dataset(config.data.data_path)
test_dataset = WikiTQDataset(dataset, config, "test")

In [ ]:
model = EEDBartModelForGenerativeQuestionAnswering(config)
model.load_state_dict(torch.load("logs/table_question_answering_tapex_eed_with_kldiv_flattened_reg_low_weight/checkpoints/epoch=30.pt"))

In [ ]:
tokenizer = test_dataset.tokenizer

In [ ]:
model.to("cuda:0")

In [ ]:
def predict(index):

    input_ids, attention_mask, token_type_ids, decoder_input_ids, labels = test_dataset.__getitem__(index)
    actual_output_ids = labels.clone()
    

    token_scores = model.model.model.sigmoid(model.model.model.token_classifier(model.model.model.decomposer(input_ids = input_ids.unsqueeze(0).to("cuda:0"), attention_mask = attention_mask.unsqueeze(0).to("cuda:0"))[0]))
    # token_scores = model.model.model.token_classifier(model.model.model.decomposer(input_ids = input_ids.unsqueeze(0).to("cuda:0"), attention_mask = attention_mask.unsqueeze(0).to("cuda:0"))[0])
    # print(token_scores)

    print(tokenizer.decode(input_ids.squeeze(0), skip_special_tokens=True))
    print("\n\n")

    input_ids = input_ids.detach().cpu()[token_scores.squeeze().detach().cpu() >= torch.mean(token_scores.detach().cpu())]
    print(tokenizer.decode(input_ids.squeeze(0), skip_special_tokens=True))
    # output_ids = model.model.generate(input_ids = input_ids.unsqueeze(0).to("cuda:0"), max_new_tokens = config.tokenizer.output_max_length, 
    #                                                         num_beams = 3, early_stopping = True, attention_mask = attention_mask.unsqueeze(0).to("cuda:0"))

    # predicted_sequence = tokenizer.decode(output_ids.squeeze(), skip_special_tokens=True)
    # actual_sequence = tokenizer.decode(actual_output_ids, skip_special_tokens = True)

    # print(predicted_sequence)
    # print(actual_sequence)

In [ ]:
predict(11)